In [1]:
from langchain_community.document_loaders import UnstructuredURLLoader

url = ['https://explodingtopics.com/blog/new-chatgpt-release-date',
       'https://medium.com/@l1g3nd/gpt-5-revealed-features-release-details-and-what-it-means-for-the-future-fb098c2a57de#:~:text=Expected%20to%20debut%20in%20early,to%20be%20a%20game%2Dchanger.',
       'https://www.pcguide.com/apps/gpt-5-release-date-price-and-features/'
       ]

loader = UnstructuredURLLoader(urls=url)
data = loader.load()

In [2]:
len(data)


3

In [3]:
data

[Document(metadata={'source': 'https://explodingtopics.com/blog/new-chatgpt-release-date'}, page_content='When Will ChatGPT-5 Be Released (Latest Info)\n\nby Josh Howarth November 26, 2024\n\nIt’s been a few months since the release of ChatGPT-4o, the most capable version of ChatGPT yet. However, GPT-5 might be just around the corner.\n\nOpenAI, the company behind ChatGPT, hasn’t publicly announced a release date for GPT-5.\n\nNevertheless, various clues — including interviews with Open AI CEO Sam Altman — indicate that GPT-5 could launch quite soon.\n\nIn this article, we’ll analyze these clues to estimate when ChatGPT-5 will be released. We’ll also discuss just how much more powerful the new AI tool will be compared to previous versions.\n\nChatGPT-5 Release Date\n\nAccording to the latest available information, ChatGPT-5 is set to be released sometime in late 2024 or early 2025.\n\nThis estimate is based on public statements by OpenAI, interviews with Sam Altman, and timelines of pr

so now to divide our data into smaller , why do we need to divide our data into chunks ?
- helps maintain the context of the text
- reducing computational and improving overall performance
- allows for more efficient indexing and retrieval of relevant information
- better overall summaries

and why it is useful for doing chunking ? because when user will ask questions, it will only give that related answer by searching around

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

#  tiktoken library for more precise token counting when splitting text
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,  # This will now be in tokens, not characters
    chunk_overlap=0
)
docs = text_splitter.split_documents(data)

# total number of splitted documents 
print("Total number of documents:", len(docs))


Total number of documents: 26


In [5]:
docs[5]


Document(metadata={'source': 'https://explodingtopics.com/blog/new-chatgpt-release-date'}, page_content='On the other hand, there’s really no limit to the number of issues that safety testing could expose. Delays necessitated by patching vulnerabilities and other security issues could push the release of GPT-5 well into 2025.\n\nThe uncertainty of this process is likely why OpenAI has so far refused to commit to a release date for GPT-5.\n\nChatGPT-5 Features\n\nAltman and OpenAI have also been somewhat vague about what exactly ChatGPT-5 will be able to do. That’s probably because the model is still being trained and its exact capabilities are yet to be determined.\n\nHowever, Altman has hinted at some of the things GPT-5 will be capable of.\n\nWhat is the Capability of GPT-5?\n\n“It’s not like this model is going to get a little bit better,” Altman said at the World Governments Summit in Dubai in February. “It’s going to be better across the board.”\n\nGenerally, then, GPT-5 should be

Jetzt fange mit Embeddings an. warum brauchen wir das ?
- because we want to convert our text data into numerical representation 

In [6]:
# # embedding models 
from sentence_transformers import SentenceTransformer
import tqdm

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device="cpu")
docs_text = [doc.page_content for doc in docs]  # Extract text
embeddings = model.encode(docs_text)


## only to check whether the above is working on not 
vector = model.encode("Hello, Servus") # einfach beispiel 
vector[:5]

/home/6082/.RAG/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


array([-0.01166673,  0.05221786,  0.06761146,  0.01808612, -0.04677952],
      dtype=float32)

In [7]:
import faiss
import numpy as np

embeddings_array = np.array(embeddings, dtype="float32")

# Step 1: Initialize FAISS index
embedding_dim = embeddings_array.shape[1]  # Dimension of embeddings
index = faiss.IndexFlatL2(embedding_dim)   # L2 (Euclidean) distance index

# Step 2: Add embeddings to the index
index.add(embeddings_array)
print(f"Number of embeddings added to the index: {index.ntotal}")

Number of embeddings added to the index: 26


In [8]:
# Step 1: Generate embedding for the query
query_vector = model.encode("Nürnberg").astype("float32")

# Step 2: Perform the search
k = 3 # Number of nearest neighbors to retrieve
distances, indices = index.search(np.array([query_vector]), k)

# Step 3: Retrieve corresponding documents
retrieved_docs = [docs_text[idx] for idx in indices[0]]

print("Retrieved Documents:")
for doc in retrieved_docs:
    print(doc)


Retrieved Documents:
GPT-4, OpenAI’s current flagship AI model, is now a mature foundation model. With GPT-4V and GPT-4 Turbo released in Q4 2023, the firm ended last year on a strong note. However, there has been little in the way of official announcements from OpenAI on their next version, despite industry experts assuming a late 2024 arrival.

Previously, at an MIT event in April 2023, OpenAI CEO Sam Altman shared that the company was not training GPT-5 at that time. However, since then, the chief executive has sought new funding from key investor Microsoft to build a new superintelligence. Sharing the stage with Satya Nadella at OpenAI DevDay, the firm’s first developer conference, Altman reminisced “I think we have the best partnership in tech. I’m excited for us to build AGI together.”
But does this mean just another incremental improvement to AI tools, or does it mean the arrival of AGI?

AGI, or artificial general intelligence, is the concept of machine intelligence on par with

In [9]:
from langchain_huggingface import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
from langchain_core.output_parsers import StrOutputParser
import torch 

In [10]:
model_id = "distilbert/distilgpt2"

In [11]:
text_generation_pipeline = pipeline(
    "text-generation",model=model_id,torch_dtype=torch.float32 , max_new_tokens=200, device="cpu")

Device set to use cpu


In [12]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [13]:
prompt_template = """
<|system|>
Use the following context to answer the question.

Context: 
{context}

Question: 
{question}

Answer:

"""

In [14]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template= prompt_template
)

In [15]:
from langchain.chains import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm, output_parser=StrOutputParser())

# so how does this llm chain works 
# '''
# first user will ask some question and then that question goes to the llm model so that It will search the answer for and then we will receive a ouput in more readable form with the help of StrOutputParser
# '''


/tmp/ipykernel_938260/3322413079.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm, output_parser=StrOutputParser())


In [16]:
context = "\n".join(retrieved_docs)

In [17]:
# Step 1: Generate embedding for the query
query = "When will gpt5 release in germany?"
query_vector = model.encode(query).astype("float32")

In [18]:
question = query
llm_input = {"context": context, "question": question}

In [19]:
response = llm_chain.run(llm_input)


/tmp/ipykernel_938260/2927182111.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run(llm_input)
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [20]:
print(response)


<|system|>
Use the following context to answer the question.

Context: 
GPT-4, OpenAI’s current flagship AI model, is now a mature foundation model. With GPT-4V and GPT-4 Turbo released in Q4 2023, the firm ended last year on a strong note. However, there has been little in the way of official announcements from OpenAI on their next version, despite industry experts assuming a late 2024 arrival.

Previously, at an MIT event in April 2023, OpenAI CEO Sam Altman shared that the company was not training GPT-5 at that time. However, since then, the chief executive has sought new funding from key investor Microsoft to build a new superintelligence. Sharing the stage with Satya Nadella at OpenAI DevDay, the firm’s first developer conference, Altman reminisced “I think we have the best partnership in tech. I’m excited for us to build AGI together.”
But does this mean just another incremental improvement to AI tools, or does it mean the arrival of AGI?

AGI, or artificial general intelligence

In [21]:
response = response.replace("</s>", "").strip()
print("Antwort", response)

Antwort <|system|>
Use the following context to answer the question.

Context: 
GPT-4, OpenAI’s current flagship AI model, is now a mature foundation model. With GPT-4V and GPT-4 Turbo released in Q4 2023, the firm ended last year on a strong note. However, there has been little in the way of official announcements from OpenAI on their next version, despite industry experts assuming a late 2024 arrival.

Previously, at an MIT event in April 2023, OpenAI CEO Sam Altman shared that the company was not training GPT-5 at that time. However, since then, the chief executive has sought new funding from key investor Microsoft to build a new superintelligence. Sharing the stage with Satya Nadella at OpenAI DevDay, the firm’s first developer conference, Altman reminisced “I think we have the best partnership in tech. I’m excited for us to build AGI together.”
But does this mean just another incremental improvement to AI tools, or does it mean the arrival of AGI?

AGI, or artificial general intel

#### Another model just to compare 